In [1]:
pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 3.0 MB/s eta 0:00:00


In [2]:
pip install spacy_langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.1/998.1 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993416 sha256=0cfde08b8d42bf907f1dc35d885ff615d4af6248cb0812b1d5aa052e60226d19
  Stored in directory: /root/.cache/pip/wheels/97/f1/e4/8b73f7a0421b132755956892d29b1e764d3e0857a6e92e32fe
Successfully built langdetect


In [3]:
import praw
import pandas as pd
import seaborn as sns
import numpy as np

In [4]:
import re    # RegEx for removing non-letter characters
import nltk  #natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
text = 'Hola que tal, como estas'
doc = nlp(text)
print(doc._.language)

{'language': 'es', 'score': 0.9999936494944687}


In [ ]:
r = doc._.language["language"]
print(r, type(r))

es <class 'str'>


In [7]:
reddit = praw.Reddit(client_id='TlPT-9dolaABWzzGIxim8Q', client_secret='KcSNwe14od5UfnHs-Ps5mBfdPYm6RA', \
                     user_agent='Jenifer Vivar', check_for_async=False)


In [8]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [9]:
nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

NameError: ignored

In [14]:

posts = []
ml_subreddit = reddit.subreddit('moral')
for post in ml_subreddit.hot(limit=50):

  doc = nlp(post.selftext)
  language_ = doc._.language['language']
  
  if language_ == "en":
    text_words = tweet_to_words(post.selftext)
    posts.append([post.title, post.id, post.subreddit, post.url, post.num_comments, post.selftext, text_words])
  else:
    pass

posts = pd.DataFrame(posts,columns=['title', 'id', 'subreddit', 'url', 'num_comments', 'body', "lemmatized_text"])


In [15]:
posts.head()

,title,id,subreddit,url,num_comments,body,lemmatized_text
0,Staring at the beach,13a21mf,moral,https://www.reddit.com/r/moral/comments/13a21m...,2,"Is it wrong that while on the beach,i can't he...","[wrong, beach, help, notic, lust, ladi, show, ..."
1,Is it ok to kill a rapist?,12i476q,moral,https://www.reddit.com/r/moral/comments/12i476...,5,I really wanna kill this asshole dude who does...,"[realli, wanna, kill, asshol, dude, care, girl..."
2,Age gap relationship,12fcsmj,moral,https://www.reddit.com/r/moral/comments/12fcsm...,1,I had trouble posting this in the relationship...,"[troubl, post, relationship, one, go, realli, ..."
3,Is stealing from a store morally incorrect?,126bznn,moral,https://www.reddit.com/r/moral/comments/126bzn...,9,I feel like stealing some crap from a gas stat...,"[feel, like, steal, crap, ga, station, someth,..."
4,Morality of designing/making weapons as a job,11taxxb,moral,https://www.reddit.com/r/moral/comments/11taxx...,0,Would you consider the many kinds of engineers...,"[would, consid, mani, kind, engin, moral, choo..."


In [16]:
len(posts)

30

In [21]:
30*300

9000

In [31]:
comments_dict = {
  "comment_body" : [],
  "comment_link_id" : [],
  "comment_id" : [],  
      
  }
  
for url_ in posts["url"]:


  submission = reddit.submission(url=url_)
  #submission.comments.replace_more(limit = 300)
  i =0
  for comment in submission.comments.list():
    if i <=300:
      comments_dict["comment_body"].append(comment.body)
      comments_dict["comment_link_id"].append(comment.link_id)
      comments_dict["comment_id"].append(comment.id)
      i+=1
    

    
  


In [32]:
post_comments_ = pd.DataFrame(comments_dict)
post_comments_.head()

,comment_body,comment_link_id,comment_id
0,"I mean yeah, you shouldn’t stare at people’s b...",t3_13a21mf,jj8yroq
1,I guess in mind I was trying to rationalize it...,t3_13a21mf,jjaq7p2
2,Yes.,t3_12i476q,jfsxoh9
3,Okay then how about you kill yourself first th...,t3_12i476q,jfxwk0y
4,Yes killing a rapist is ok killing that guy wh...,t3_12i476q,jh44p4h


In [33]:
len(post_comments_)

43

In [ ]:

post_comments_.to_csv("/content/drive/MyDrive/den_yelp_notebooks/commentsHot_subreddit.csv")

In [35]:
sub = ["NoStupidQuestions"]  # make a list of subreddits you want to scrape the data from

for s in sub:
    subreddit = reddit.subreddit(s)
    

In [39]:

query = ['moral' 'ethics', 'values']
dfs_comments = []
dfs_data = []
for item in query:

  post_dict = {
      "title" : [],   #title of the post
      "score" : [],   # score of the post
      "id" : [],      # unique id of the post
      "url" : [],     #url of the post
      "comms_num": [],   #the number of comments on the post
      "created" : [],  #timestamp of the post
      "body" : []         # the descriptionof post
  }
  comments_dict = {
      "comment_id" : [],      #unique comm id
      "comment_parent_id" : [],   # comment parent id
      "comment_body" : [],   # text in comment
      "comment_link_id" : [],
      "comment_important_words":[] #link to the comment
       
  }
  
  for submission in subreddit.search(query, sort = "top",limit = 10):
      post_dict["title"].append(submission.title)
      post_dict["score"].append(submission.score)
      post_dict["id"].append(submission.id)
      post_dict["url"].append(submission.url)
      post_dict["comms_num"].append(submission.num_comments)
      post_dict["created"].append(submission.created)
      post_dict["body"].append(submission.selftext)
      
      ##### Acessing comments on the post
      submission.comments.replace_more(limit = 1)
      for comment in submission.comments.list():
        comment_ = tweet_to_words(comment.body)
        comments_dict["comment_id"].append(comment.id)
        comments_dict["comment_parent_id"].append(comment.parent_id)
        comments_dict["comment_body"].append(comment.body)
        comments_dict["comment_important_words"].append(comment_)
        comments_dict["comment_link_id"].append(comment.link_id)
        

  post_comments = pd.DataFrame(comments_dict)
  dfs_comments.append(post_comments)
  #post_comments.to_csv(s+"_comments_"+ item +"subreddit.csv")
  post_data = pd.DataFrame(post_dict)
  dfs_data.append(post_data)
  #post_data.to_csv(s+"_"+ item +"subreddit.csv")



In [42]:
df_concat_comments = pd.concat(dfs_comments, axis=0)
df_concat_comments.head()
df_concat_comments.to_csv("/content/drive/MyDrive/den_yelp_notebooks/comments_subreddit2.csv")

In [43]:
len(df_concat_comments)

8790

In [44]:
df_concat_comments["comment_body"].iloc[7]

'I don’t get it. If you were supposed to get $12, but recieved a single bill wouldn’t you logically unfold the bill to see how much change you’d need to give back? I mean whatever amount that bill was it was not going to be the correct amount, no?'

In [45]:
len(df_concat_comments)

8790

In [46]:
df_metadata = pd.concat(dfs_data, axis=0)
df_metadata.head()


,title,score,id,url,comms_num,created,body
0,Delivered $12 food order to a person with Down...,20062,l2d2ba,https://www.reddit.com/r/NoStupidQuestions/com...,1306,1.611280e+09,I have a job delivering for a local sandwich s...
1,Are NFTs like beanie babies where it's somethi...,15610,qonpbp,https://www.reddit.com/r/NoStupidQuestions/com...,2035,1.636289e+09,
2,Is it insensitive of me for not to want religi...,11124,wx43fo,https://www.reddit.com/r/NoStupidQuestions/com...,1285,1.661400e+09,I'm 19 and I'm struggling with my mental healt...
3,"If an astronaut dies while in space, do they e...",10216,gsffgv,https://www.reddit.com/r/NoStupidQuestions/com...,498,1.590702e+09,I'm asking because I know some cultures really...
4,If its illegal to sell a house to your buddy f...,4007,13j0bcr,https://www.reddit.com/r/NoStupidQuestions/com...,641,1.684230e+09,


In [47]:
len(df_metadata)

20

In [48]:
df_metadata.to_csv("/content/drive/MyDrive/den_yelp_notebooks/metadata_subreddit2.csv")

In [ ]:
df_metadata["title"].value_counts()

I can’t breathe. Black lives matter.                                                                                                                          5
How do you feel about ’how would you feel?’ posts being banned?                                                                                               5
Where can you find fair, honest and reasonably unbiased opinions on world news and politics?                                                                  5
Reddit, Can we have a reddit job fair?                                                                                                                        5
Mother-In-Laws of Reddit, in the interest of fairness, what are your crazy daughter-in-law stories?                                                           5
You die. Death himself however says if you can beat him at a fair game of your choice, you get a second chance at life. What game do you challenge him to?    5
Who is the best example of someone that 

In [ ]:
for title in df_metadata["title"]:
  print(title)

I can’t breathe. Black lives matter.
How do you feel about ’how would you feel?’ posts being banned?
Where can you find fair, honest and reasonably unbiased opinions on world news and politics?
Reddit, Can we have a reddit job fair?
Mother-In-Laws of Reddit, in the interest of fairness, what are your crazy daughter-in-law stories?
You die. Death himself however says if you can beat him at a fair game of your choice, you get a second chance at life. What game do you challenge him to?
Who is the best example of someone that you're not sure if they are a complete idiot or a genius?
[UPDATE] I was requested to compile all of the real-life cheats into an easy to read list.
Reddit, what's a fairly unknown yet super interesting subreddit that will pass the time?
Reddit, what is the most fucked up way you got back at someone that wronged you. *Throwaways encouraged*
I can’t breathe. Black lives matter.
How do you feel about ’how would you feel?’ posts being banned?
Where can you find fair, hon

In [ ]:
len(post_comments)

593

In [ ]:
post_data

,title,score,id,url,comms_num,created,body
0,what scientific experiment would you run if mo...,73956,e31v9o,https://www.reddit.com/r/AskReddit/comments/e3...,19529,1.574965e+09,


In [ ]:
dfs_data[1]

,title,score,id,url,comms_num,created,body
0,what scientific experiment would you run if mo...,73954,e31v9o,https://www.reddit.com/r/AskReddit/comments/e3...,19529,1.574965e+09,


In [ ]:

query = ['moral',"Morality", "ethical"]
dfs_comments = []
dfs_data = []
for item in query:

  post_dict = {
      "title" : [],   #title of the post
      "score" : [],   # score of the post
      "id" : [],      # unique id of the post
      "url" : [],     #url of the post
      "comms_num": [],   #the number of comments on the post
      "created" : [],  #timestamp of the post
      "body" : []         # the descriptionof post
  }
  comments_dict = {
      "comment_id" : [],      #unique comm id
      "comment_parent_id" : [],   # comment parent id
      "comment_body" : [],   # text in comment
      "comment_link_id" : [],
      "comment_important_words":[]  #link to the comment
  }
  result 
  for submission in subreddit.search(query,sort = "top",limit = 3000):
      post_dict["title"].append(submission.title)
      post_dict["score"].append(submission.score)
      post_dict["id"].append(submission.id)
      post_dict["url"].append(submission.url)
      post_dict["comms_num"].append(submission.num_comments)
      post_dict["created"].append(submission.created)
      post_dict["body"].append(submission.selftext)
      
      ##### Acessing comments on the post
      submission.comments.replace_more(limit = 1)
      for comment in submission.comments.list():
        comment_ = tweet_to_words(comment.body)
        comments_dict["comment_id"].append(comment.id)
        comments_dict["comment_parent_id"].append(comment.parent_id)
        comments_dict["comment_body"].append(comment.body)
        comments_dict["comment_important_words"].append(comment_)
        comments_dict["comment_link_id"].append(comment.link_id)
  
  post_comments = pd.DataFrame(comments_dict)
  dfs_comments.append(post_comments)
  #post_comments.to_csv(s+"_comments_"+ item +"subreddit.csv")
  post_data = pd.DataFrame(post_dict)
  dfs_data.append(post_data)
  #post_data.to_csv(s+"_"+ item +"subreddit.csv")

